In [8]:
## Credit Risk Classification Problem
# Download appropriate libraries for the project
# Fundamentals
import numpy as np # Fundamental package for Python computing 
import pandas as pd # Easy to use data analysis and structuring

# Plotting
import seaborn as sns
colour = sns.color_palette()
import matplotlib 
import matplotlib.pyplot as plt

# Modelling and processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
import datetime

In [11]:
# Load Data
df_customer = pd.read_csv("customer_data.csv")
df_payment = pd.read_csv("payment_data.csv")

In [12]:
# Inspect head and dimensions to check import
df_customer.head()

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
0,1,54982665,5,1245.5,3,77000.0,2,15,5,109,5,151300,244.948974
1,0,59004779,4,1277.0,1,113000.0,2,8,-1,100,3,341759,207.173840
2,0,58990862,7,1298.0,1,110000.0,2,11,-1,101,5,72001,1.000000
3,1,58995168,7,1335.5,1,151000.0,2,11,5,110,3,60084,1.000000
4,0,54987320,7,NaN,2,59000.0,2,11,5,108,4,450081,197.403141


In [16]:
print("Dimensions of Customer data:", df_customer.shape)

Dimensions of Customer data: (1125, 13)


In [13]:
df_payment.head()

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,NaN,NaN
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,491100.0,NaN
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,700500.0,22/04/2016
3,54988608,0,0,0,0,3,10,37400.0,03/12/2016,8425.2,7520.0,25/04/2016
4,54987763,0,0,0,0,2,10,NaN,03/12/2016,15147.6,NaN,26/04/2016


In [17]:
print("Dimensions of Payment data:", df_payment.shape)

Dimensions of Payment data: (8250, 12)


In [18]:
# Overview of summary statistics for each dataset
df_customer.describe(include='all')

,label,id,fea_1,fea_2,fea_3,fea_4,fea_5,fea_6,fea_7,fea_8,fea_9,fea_10,fea_11
count,1125.000000,1.125000e+03,1125.000000,976.000000,1125.000000,1.125000e+03,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000,1125.000000
mean,0.200000,5.783677e+07,5.482667,1283.911373,2.333333,1.208836e+05,1.928889,10.872000,4.832889,100.802667,4.195556,164618.495111,134.999004
std,0.400178,1.817150e+06,1.383338,51.764022,0.878773,8.844523e+04,0.257125,2.676437,2.971182,11.988955,0.855679,152520.488281,112.616798
min,0.000000,5.498235e+07,1.000000,1116.500000,1.000000,1.500000e+04,1.000000,3.000000,-1.000000,64.000000,1.000000,60000.000000,1.000000
25%,0.000000,5.499050e+07,4.000000,1244.000000,1.000000,7.200000e+04,2.000000,8.000000,5.000000,90.000000,3.000000,60044.000000,1.000000
50%,0.000000,5.898975e+07,5.000000,1281.500000,3.000000,1.020000e+05,2.000000,11.000000,5.000000,105.000000,4.000000,72000.000000,173.205081
75%,0.000000,5.899799e+07,7.000000,1314.500000,3.000000,1.390000e+05,2.000000,11.000000,5.000000,111.000000,5.000000,151307.000000,202.484567
max,1.000000,5.900624e+07,7.000000,1481.000000,3.000000,1.200000e+06,2.000000,16.000000,10.000000,115.000000,5.000000,650070.000000,707.106781


In [20]:
df_payment.describe(include='all')

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
count,8.250000e+03,8250.000000,8250.000000,8250.000000,8250.000000,8250.000000,8250.000000,2132.000000,8224,8.250000e+03,7.841000e+03,7136
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3041,NaN,NaN,1862
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19/05/2015,NaN,NaN,21/09/2015
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,NaN,NaN,103
mean,5.782173e+07,0.249091,0.127152,0.369212,187.681697,14.526667,8.232000,85789.702205,NaN,1.054042e+05,2.192027e+05,NaN
std,1.822724e+06,1.250197,0.860046,2.900320,1804.232570,12.053627,3.533055,74345.828184,NaN,1.887704e+06,2.814536e+06,NaN
min,5.498235e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.100000,NaN,-4.030320e+04,5.010000e+02,NaN
25%,5.499050e+07,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,37400.000000,NaN,0.000000e+00,2.345300e+04,NaN
50%,5.898905e+07,0.000000,0.000000,0.000000,0.000000,11.000000,10.000000,68200.000000,NaN,0.000000e+00,4.404700e+04,NaN
75%,5.899655e+07,0.000000,0.000000,0.000000,0.000000,25.000000,10.000000,112200.000000,NaN,2.494800e+04,1.005000e+05,NaN


In [23]:
# Check if we have missing data
df_customer.isnull().any()

label     False
id        False
fea_1     False
fea_2      True
fea_3     False
fea_4     False
fea_5     False
fea_6     False
fea_7     False
fea_8     False
fea_9     False
fea_10    False
fea_11    False
dtype: bool

In [25]:
df_payment.isnull().any()

id                 False
OVD_t1             False
OVD_t2             False
OVD_t3             False
OVD_sum            False
pay_normal         False
prod_code          False
prod_limit          True
update_date         True
new_balance        False
highest_balance     True
report_date         True
dtype: bool

In [40]:
# We know we have some missing data issues, lets ivnestigate the severity further
# Here we define a function that will report both the instances and proprtion of missings per column
def get_missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False) # This sums "TRUE" of isnull and sorts values most to fewest
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False) # Calculate the proportion of missings
    percent = round(percent, 2)
    # Now combine these two bits of info i.) Total, ii.) Percent into a datafram to print
    missing_df = pd.concat([total,percent], axis=1, keys=["Total", "Percent"])
    print(missing_df.head(20))
    return

In [42]:
get_missing_data(df_customer)

        Total  Percent
fea_2     149    13.24
fea_11      0     0.00
fea_10      0     0.00
fea_9       0     0.00
fea_8       0     0.00
fea_7       0     0.00
fea_6       0     0.00
fea_5       0     0.00
fea_4       0     0.00
fea_3       0     0.00
fea_1       0     0.00
id          0     0.00
label       0     0.00


In [43]:
get_missing_data(df_payment)

                 Total  Percent
prod_limit        6118    74.16
report_date       1114    13.50
highest_balance    409     4.96
update_date         26     0.32
new_balance          0     0.00
prod_code            0     0.00
pay_normal           0     0.00
OVD_sum              0     0.00
OVD_t3               0     0.00
OVD_t2               0     0.00
OVD_t1               0     0.00
id                   0     0.00
